In [ ]:
!pip install torchaudio==0.13.0 torchvision==0.14.0 torch==1.13.0

In [ ]:
!pip install openmim==0.3.2
!mim install mmcv-full==1.7.0
!pip install -e .

In [ ]:
!pip install -r requirements.txt 

In [ ]:
!apt install libgl1-mesa-glx --yes

In [ ]:
!pip install mmdet==2.27.0

In [24]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from mmdet.models import build_detector
from mmdet.apis import init_detector, inference_detector
from mmcv import Config

In [25]:
model_path = "best_bbox_h_mAP_epoch_33.pth" 
config_path = "deformable_detr_config.py"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = init_detector(config_path, model_path, device=device)

cuda
load checkpoint from local path: best_bbox_h_mAP_epoch_33.pth


In [ ]:
frame = cv2.imread("20241203_170425.jpg")  # test frame
results = inference_detector(model, frame)
print(f"Inference results: {results}")


In [49]:
import cv2
cap = cv2.VideoCapture("output.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

video = []

while True:    
    rt, frame = cap.read()
    if not rt:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (640, 480))

    video.append(frame_resized)

#print(video)

30.0


In [ ]:
from time import sleep
from tqdm import tqdm
import cv2

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
saved_video = cv2.VideoWriter("saved_video.mp4", fourcc, fps, (640, 480))

for frame_resized in tqdm(video):
    results = inference_detector(model, frame_resized)

    #print(f"Results length: {len(results)}")
    for idx, result in enumerate(results):
        #print(f"Processing result {idx+1}/{len(results)}")
        for bbox in result:
            #print("Bounding box:", bbox)
            x_min, y_min, x_max, y_max, score = bbox
            if score > 0.6:  # Confidence threshold
                label = "Gun Detected"
                #print(label)
                cv2.rectangle(frame_resized, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                #cvzone.cornerRect(frame, [int(x1), int(y1), int(x2-x1), int(y2-y1)], l=9, rt=3)
                cv2.putText(frame_resized, label, (int(x_min), int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    
    frame_resized = frame_resized[:, :, ::-1]
    saved_video.write(frame_resized)

saved_video.release()
cv2.destroyAllWindows()

print("done")

100%|██████████| 2606/2606 [02:10<00:00, 20.04it/s]

done
